In [ ]:
import pandas as pd

In [ ]:
googleUsers = pd.read_parquet('../datasets/processed/google/reviews_google.parquet.gz')

In [ ]:
usuarios = googleUsers['user_id'].unique()

In [ ]:
usuariosG = googleUsers.drop(columns=['gmap_id', 'resp_time', 'state', 'sentiment_textblob_resp', 'sentiment'])

In [ ]:
usuariosG[['user_id','name']].drop_duplicates()

In [ ]:
# Sacar milesimas de segundo
usuariosG['time'] = usuariosG['time'].astype(str)
usuariosG['time'] = usuariosG['time'].apply(lambda x: x.split('.')[0])
usuariosG['time'] = pd.to_datetime(usuariosG['time'])
usuariosG